In [1]:
import cvxpy as cp
from cvxpygen import cpg
import numpy as np
import numba
import cffi
from numba.core.typing import cffi_utils
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("../optimisation/")
import plasticity_framework as pf
import convex_return_mapping as crm

In [3]:
Pa_dim = 70e3
E = 70e3 / Pa_dim #[-]
nu = 0.3 #[-]

sig0 = 250 / Pa_dim #[-]
Et = E/100.  # tangent modulus
H = E*Et/(E-Et)  # hardening modulus

vonMises = crm.vonMises(sig0, H)
material_vM = crm.Material(crm.IsotropicElasticity(E, nu), vonMises)
alpha = 1
DruckerPrager = crm.DruckerPrager(sig0, alpha, H)
material_DP = crm.Material(crm.IsotropicElasticity(E, nu), DruckerPrager)


In [4]:
return_mapping = crm.ReturnMapping(material_DP, 3)

# CVXPYgen

In [5]:
code_dir = 'code_dir'

In [6]:
# %%capture
cpg.generate_code(return_mapping.opt_problem, code_dir=code_dir, solver='SCS')

Generating code with CVXPYgen ...
CVXPYgen finished generating code.
Compiling python wrapper with CVXPYgen ... 
-- The C compiler identification is GNU 11.2.0
-- The CXX compiler identification is GNU 11.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Setting build type to 'Release' as none was specified.
-- Single precision floats (32bit) are OFF
-- Long integers (64bit) are OFF
-- COMPILER_OPTS = -DUSE_LAPACK -DCTRLC
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/work_folder/convex-plasticity/cvxpygen_custom/code_dir/c/build
Scanning dependencies of target cpg
[  0%] Building C object CMa

/mnt/work_folder/convex-plasticity/cvxpygen_custom/code_dir/c/solver_code/src/rw.c: In function ‘_scs_read_data’:
/mnt/work_folder/convex-plasticity/cvxpygen_custom/code_dir/c/solver_code/src/rw.c:184:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  184 |   fread(&(file_int_sz), sizeof(uint32_t), 1, fin);
      |   ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/mnt/work_folder/convex-plasticity/cvxpygen_custom/code_dir/c/solver_code/src/rw.c:185:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  185 |   fread(&(file_float_sz), sizeof(uint32_t), 1, fin);
      |   ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/mnt/work_folder/convex-plasticity/cvxpygen_custom/code_dir/c/solver_code/src/rw.c:202:3: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  202 |   fread(&(file_version_sz), sizeof(uint32_t), 1, fin);


[ 31%] Building C object CMakeFiles/cpg.dir/solver_code/src/scs_version.c.o
[ 34%] Building C object CMakeFiles/cpg.dir/solver_code/src/util.c.o
[ 37%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/csparse.c.o
[ 41%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/scs_matrix.c.o
[ 44%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/cpu/direct/private.c.o
[ 48%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/qdldl/qdldl.c.o
[ 51%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/SuiteSparse_config.c.o
[ 55%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_1.c.o
[ 58%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_2.c.o
[ 62%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_aat.c.o
[ 65%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/amd_control.c.o
[ 68%] Building C object CMakeFiles/cpg.dir/solver_code/linsys/external/amd/a

# CFFI

In [10]:
with open('cffi_wrapper/cffi_wrapper.c', 'r') as file:
    cdef = file.read()

with open('cffi_wrapper/cffi_wrapper.h', 'r') as file:
    source = file.read()
    
lib_dir = os.path.join(os.getcwd(), code_dir + '/c/build/out')
solver_include_dir = os.path.join(os.getcwd(), code_dir + '/c/solver_code/include')
include_dir = os.path.join(os.getcwd(), code_dir + '/c/include')

ffibuilder = cffi.FFI()

ffibuilder.set_source(
  module_name='_cpglib', 
  source=source,
  include_dirs = [include_dir, solver_include_dir],
  libraries = ['cpg'],
  library_dirs = [lib_dir],
)

ffibuilder.cdef(csource=cdef)

ffibuilder.compile()

'/mnt/work_folder/convex-plasticity/cvxpygen_custom/_cpglib.cpython-39-x86_64-linux-gnu.so'

# CFFI + Numba

In [11]:
import _cpglib

cffi_utils.register_module(_cpglib)

CPG_Updated_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Updated_cpp_t'), use_record_dtype=True)
CPG_Params_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Params_cpp_t'), use_record_dtype=True)
CPG_Result_python_t = cffi_utils.map_type(_cpglib.ffi.typeof('CPG_Result_cpp_t'), use_record_dtype=True)

cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Updated_cpp_t'), CPG_Updated_python_t)
cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Params_cpp_t'), CPG_Params_python_t)
cffi_utils.register_type(_cpglib.ffi.typeof('CPG_Result_cpp_t'), CPG_Result_python_t)

solve_signature = cffi_utils.map_type(_cpglib.ffi.typeof(_cpglib.lib.solve_cpp), use_record_dtype=True)

In [12]:
@numba.cfunc(solve_signature, nopython=True)
def solve_wrapper(upd, par, res):
    _cpglib.lib.solve_cpp(upd, par, res)

cpg_sol = solve_wrapper.ctypes
cpg_sol

In [13]:
#Preliminary initialization of parameters, functions, etc (before the numba-function invocation)

upd = _cpglib.ffi.new("CPG_Updated_cpp_t *", {})
par = _cpglib.ffi.new("CPG_Params_cpp_t *", {})
res = _cpglib.ffi.new("CPG_Result_cpp_t *", {})

upd_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(upd), dtype=numba.np.numpy_support.as_dtype(CPG_Updated_python_t), shape=1,)
par_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(par), dtype=numba.np.numpy_support.as_dtype(CPG_Params_python_t), shape=1,)
res_numpy = np.ndarray(buffer=_cpglib.ffi.buffer(res), dtype=numba.np.numpy_support.as_dtype(CPG_Result_python_t), shape=1,)

updated_params = ['sig_old', 'deps', 'p_old']
for p in updated_params:
    setattr(upd, p, True)

_cpglib.lib.cpg_set_solver_default_settings()
_cpglib.lib.cpg_set_solver_verbose(False)


In [11]:
@numba.njit
def call_solver(upd_numpy: np.ndarray, par_numpy: np.ndarray, res_numpy: np.ndarray, Sig: np.ndarray):
    for i in range(N + 1):
        par_numpy['sig_old'][:] = zeros[:]
        par_numpy['deps'][:] = Eps[i,:]

        cpg_sol(upd_numpy.ctypes.data, par_numpy.ctypes.data, res_numpy.ctypes.data)

        Sig[i,:] = res_numpy['prim']['sig'][0]


In [12]:
call_solver(upd_numpy, par_numpy, res_numpy, Sig)